In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *

import os


In [2]:
data_dir = f'e:/kaggle/05-LabelEncoded-last'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/TestC5')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'label_encoded_data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'bad_for_training_features',
           params={
               'input_directory': data_dir,
               'file': 'bad_for_training_features.yaml'
           })
p.save()
p.run(verbose=True)

---------------------------
0: LoaderNode [2019-09-30 22:31:44]
params:
 {'input_directory': 'e:/kaggle/05-LabelEncoded-last', 'file': 'label_encoded_data.pkl'}
---------------------------
1: LoaderNode [2019-09-30 22:31:59]
params:
 {'input_directory': 'e:/kaggle/05-LabelEncoded-last', 'file': 'numerical_columns.yaml'}
---------------------------
2: LoaderNode [2019-09-30 22:31:59]
params:
 {'input_directory': 'e:/kaggle/05-LabelEncoded-last', 'file': 'categorical_columns.yaml'}
---------------------------
3: LoaderNode [2019-09-30 22:31:59]
params:
 {'input_directory': 'e:/kaggle/05-LabelEncoded-last', 'file': 'bad_for_training_features.yaml'}


In [3]:
data=p.data['data']
data_slice = data[['TransactionAmt','new_card_id','Date']]

In [5]:
group_column = 'new_card_id'
num_feature = 'TransactionAmt'
#     data_slice = data[['Date',group_column,num_feature]].reset_index()
gb = data_slice.groupby([group_column])
q = gb[num_feature].diff()
# ds = data_slice.set_index([group_column, 'Date'])
# ds = ds.join(q)
# ds = ds.set_index('TransactionID').sort_index()

ValueError: cannot join with no overlapping index names

In [7]:
test_ids = data[data.new_card_id == 1000].index

In [10]:
data_slice.loc[test_ids]

,TransactionAmt,new_card_id,Date
TransactionID,,,
3372591,117.0,1000,2018-03-22 20:42:25
3387485,59.0,1000,2018-03-27 21:23:47
3387500,92.0,1000,2018-03-27 21:27:01
3440492,59.0,1000,2018-04-14 00:43:28
3446580,107.0,1000,2018-04-16 15:25:21
...,...,...,...
4107245,59.0,1000,2018-12-17 19:08:24
4109721,141.0,1000,2018-12-18 12:17:42
4113256,107.0,1000,2018-12-19 01:32:12


In [8]:
q.loc[test_ids]

TransactionID
3372591     NaN
3387485   -58.0
3387500    33.0
3440492   -33.0
3446580    48.0
           ... 
4107245     4.0
4109721    82.0
4113256   -34.0
4164391   -58.0
4164693    -4.0
Name: TransactionAmt, Length: 72, dtype: float32

In [22]:
q = q.rename(columns={'TransactionAmt':'TransactionAmt_diff_by_new_card_id'})


In [ ]:
q = pd.DataFrame(q)

In [30]:
q = q.rename(columns={0:'TransactionAmt_diff_by_new_card_id'})

In [31]:
q

,TransactionAmt_diff_by_new_card_id
TransactionID,
2987000,NaN
2987001,NaN
2987002,NaN
2987003,NaN
2987004,NaN
...,...
4170235,57.914001
4170236,-26.006001
4170237,-151.000000


In [32]:
q.to_pickle('e:/kaggle/05-LabelEncoded-last/tramt_diff.pkl')

In [13]:
test = pd.concat([q,data.isFraud],axis=1)

In [16]:
test[test.TransactionAmt == 0][test.isFraud==1]

c:\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,TransactionAmt,isFraud
TransactionID,,
2987243,0.0,1
2987245,0.0,1
2987781,0.0,1
2988195,0.0,1
2988680,0.0,1
...,...,...
3576721,0.0,1
3576891,0.0,1
3577092,0.0,1


In [18]:
test[test.isFraud==1]

,TransactionAmt,isFraud
TransactionID,,
2987203,NaN,1
2987240,NaN,1
2987243,0.000000,1
2987245,0.000000,1
2987288,NaN,1
...,...,...
3577361,-427.000000,1
3577364,21.886997,1
3577368,NaN,1


In [17]:
test[test.TransactionAmt == 0][test.isFraud==0]

c:\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,TransactionAmt,isFraud
TransactionID,,
2987104,0.0,0
2987118,0.0,0
2987125,0.0,0
2987127,0.0,0
2987144,0.0,0
...,...,...
3577514,0.0,0
3577517,0.0,0
3577518,0.0,0


In [34]:
from BayDS.lib.training import *
from sklearn.model_selection import GroupKFold

NFOLDS = 6
folds = GroupKFold(n_splits=NFOLDS)
params = {
          'objective': 'binary',
          'learning_rate': 0.05,
          "boosting_type": "gbdt",
          "metric": 'auc',
          "verbosity": -1,
          'random_state': 47
         }
train_options = {
        "model_type":'lgb',
        "params": params,
        "eval_metric":'auc',
        'early_stopping_rounds': 100,
        'n_estimators': 200,
        'averaging': 'usual',
        'use_groups': False,
        'fold_name': folds.__class__.__name__,
        'n_splits': NFOLDS
    }

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [36]:
train_idx = data[data.isFraud>=0].index
X = q.loc[train_idx]
y = data.isFraud.loc[train_idx]
test=None


result_dict = train_model_classification_vb( splits=1,
                                             X=X, X_test=test, 
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             plot_feature_importance=False,
                                             verbose=-1, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=data['DT_M'].loc[train_idx])
score = result_dict['scores'][0]
print(f'{score}')

Fold 6 started at Mon Sep 30 22:54:42 2019
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.603871	training's auc: 0.602297	valid_1's auc: 0.561943	valid_1's auc: 0.560723
CV mean score: 0.5619, std: 0.0000.
0.5619429224917377
